In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using LinearAlgebra, Plots
import MeshCat as mc 
using Test, Distributions
using Random
import ForwardDiff as FD 
using Interpolations   
import ECOS  
using ProgressMeter

In [ ]:

include(joinpath(@__DIR__,"tilted_hexrotor_dynamics_v4_wind.jl"))
include(joinpath(@__DIR__,"convex_mpc.jl"))
include(joinpath(@__DIR__,"kalman_filter.jl"))


In [ ]:
########## FUNCTIONS ###########

# converting from vector of vectors <-> matrix 
function mat_from_vec(X::Vector{Vector{Float64}})::Matrix
    # convert a vector of vectors to a matrix 
    Xm = hcat(X...)
    return Xm 
end

# Controls equilibrium point
function controls_equilibrium(u0::Vector, X̄, wind_velocity, model, ode; tol= 1e-6, max_iters= 100, verbose= false)::Vector{Vector{Float64}}

    "
        Determining the equilibrium point for controls at hover
            - use Newton's method (minimize over u for least squares cost)
            - initial guess= hover location for planar hexarotor ([(9.81*mass/n_inputs)*ones(n_inputs) for i = 1:(N-1)])
    " 

    # initialize U
    U = [zeros(eltype(u0), length(u0)) for i= 1:max_iters] #empty vector for Newton guesses
    U[1]= u0 #initialize with first guess (which is simply counteracting gravity (mg/n_inputs)

    #wind_velocity= [0.0;0.0;0.0]

    for i = (1:max_iters-1)
        
        residual= (1/2) * (ode(model, X̄, U[i], wind_velocity))'* ode(model, X̄, U[i], wind_velocity) # least squares cost function
        δfδu= FD.jacobian(du -> ode(model, X̄, du, wind_velocity), U[i]) # jacobian of f with respect to u

        ∇residual= (δfδu)' * ode(model, X̄, U[i], wind_velocity) # gradient of cost function- to solve for u, this should be zero
        ∇²residual= ((δfδu)' * δfδu) + 1e-4*(I(6)) # hessian (gauss newton version) of cost function 
        # regularization term to ensure full rank
        #println("rank hessian: ", rank(∇²residual))

        norm_residual= norm(residual)

        if verbose 
            print("iter: $i    |residual|: $norm_residual   \n")
        end

        # check against tol
        # if converged, return U[1:i]
        if norm_residual < tol
            return U[1:i]
        end

        Δu= ∇²residual\(-∇residual) # Newton step
        #println(Δu)

        # update U
        U[i+1]= U[i] + vec(Δu)
        #println(U[i+1])
    end
    error("Newton's method did not converge")
end
 
function sample_wind(mean, std_dev)
    # normal distribution with the specified mean and standard deviation
    # std_dev= zeros(3)
    # for i= 1:length(mean)
    #     std_dev[i]= [(mean[i]- (1/4)*mean[i])/0.34]
    # end

    # mean= Vector{Real}(mean) 
    # std_dev=Vector{Real}(std_dev) 
    
    normal_dist_x = Normal(mean[1], std_dev[1])
    normal_dist_y = Normal(mean[2], std_dev[2])
    normal_dist_z = Normal(mean[3], std_dev[3])

    # generate a random number from the specified distribution
    wind = [rand(normal_dist_x); rand(normal_dist_y); rand(normal_dist_z)]
    return wind
end

In [ ]:
Random.seed!(1)
#### MODEL PARAMETERS ####
Ixx= 2.331e2
Ixy= -0.7923
Ixz= 2.859
Iyx= -.7923
Iyy= 2.322e2
Iyz= 2.187
Izx= 2.859
Izy= 2.187
Izz= 4.022e2
model = (mass= 6.728,
    J= Diagonal([Ixx -Ixy -Ixz; -Iyx Iyy -Iyz; -Izx -Izy Izz]), #inertia matrix- determined from CAD
    gravity= [0,0,-9.81],
    L= 0.5375, 
    kf= 13.0,#13, # constant thrust coefficient- this was guessed based on the quadrotor dynamics (linear relationship based on mass...)
    km= 0.0245,#0.0245, # propeller drag coefficient (0.0245 for quadrotor (0.5kg)
    dt= 0.02, # frequency
    α= 0.49, # radians
    β= 0.33 # radians
)
##########

#### PROBLEM DETAILS ####
# problem size 
nx = 12 # number of states
nu = 6 # number of controls
nx_a = 15 # number of states when augemented with wind
dt = model.dt
tf = 10
t_vec = 0:dt:tf
N = length(t_vec)
N_mpc= 80 # MPC horizon
N_sim= N+ N_mpc 
t_vec_sim = 0:dt:((N_sim-1)*dt)
m= model.mass
kf= model.kf

println("N= ", N)
##########

##### generate wind #####
#wind_velocity= sample_wind(model.wind_mean, model.wind_stdev)
wind_velocity = [0.2, 0.0, 0.0]
wind_stdev= [0.05; 0.05; 0.05]
# Initialize wind parameters
wind_dynamics = zeros(nx, 3)
wind_dynamics[4, 1] = 1  # Wind effect on vx
wind_dynamics[5, 2] = 1  # Wind effect on vy
wind_dynamics[6, 3] = 1  # Wind effect on vz
#########

#### EQUILIBRIUM POINT FOR HOVER ####
# newton method stuff to find the controls equilibrium point
# hover state
X̄= [0.0; 0.0; 3.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0] #+ 0.0001*randn(nx) # add a little bit of noise to reduce numerical instabilities
# initial guess for control equilibrium point- I made this the hover thrust for the planar hexrotor
u0= [(9.81*m/(nu)); (9.81*m/(nu)); (9.81*m/(nu)); (9.81*m/(nu)); (9.81*m/(nu)); (9.81*m/(nu))] 
#u0= [0.0; 10.0; 0.0; 0.0; 0.0; 0.0] 

# I determined these experimentally
u_min= -30 * ones(nu) #-100, -250
u_max= 30 * ones(nu) #100, 250
U= controls_equilibrium(u0, X̄, [0,0,0], model, hexrotor_dynamics, tol= 1e-6, max_iters= 100, verbose= true)
Ū= U[end] # control equilibrium point! will linearize around this point
println("U equilibrium point= ", Ū)
##########

#### LINEARIZATION ####
# calculate A and B matrices (jacobians of discrete dynamics with respect to X̄ and Ū)
A= FD.jacobian(dx -> rk4(model, hexrotor_dynamics, dx, Ū, [0,0,0], dt), X̄)
B= FD.jacobian(du -> rk4(model, hexrotor_dynamics, X̄, du, [0,0,0], dt), Ū)

#A= FD.jacobian(dx -> rk4(model, hexrotor_dynamics, dx, Ū, wind_velocity, dt), X̄)
#B= FD.jacobian(du -> rk4(model, hexrotor_dynamics, X̄, du, wind_velocity, dt), Ū)


#### Define costs ####
Q = diagm([100,100,100,
10,10,10,
100,100,100,
10,10,10])

Qf = 10*Q

R = 1.0*diagm(ones(nu))

# IHLQR- using the steady state P as Qn for the MPC controller
Kinf= B'
max_iters= 1e6
P= deepcopy(Qf) #initialize p
for i= 1:max_iters
    Kinf = (R .+ B' * P * B)\  (B' * P * A)
    P_recursion = Q + A' * P * A - A' * P * B * Kinf
    
    if norm(P-P_recursion) <= 1e-6
        break
    end 
    # update P for next iteration
    P = 1*P_recursion
end
# ensure that Qn is hermitian 
Qf= 1*(P +P')/2;
##########

#### KALMAN FILTER PARAMETERS ####
# process noise covariance
#σ_w= 0.9 #0.08
#W= σ_w^2 * I(15)
W= (wind_stdev[1]^2) *I(15)

# measurement noise covariance 
σ_v= 0.9 #0.08
V= σ_v^2 * I(12)

# Kalman covariance
Σ= [zeros(15, 15) for i= 1:N_sim]
Σ[1]= zeros(15, 15)
##########


#### TESTS/CHECKS ####
# double check convergence of the cost function when the equilibrium is used  
@test norm((1/2) * (hexrotor_dynamics(model, X̄, U[end], [0,0,0]))'* hexrotor_dynamics(model, X̄, U[end], wind_velocity)) < 1e-3

# check the norm of the nonlinear disretized dynamics to make sure they match at the equilibrium point
println("norm between X̄ and discretized dynamics= ", norm(X̄- rk4(model, hexrotor_dynamics, X̄, Ū, [0,0,0], dt)))
@assert norm(X̄- rk4(model, hexrotor_dynamics, X̄, Ū, [0,0,0], dt)) < 1e-3

# check for system stability
eig= abs.(eigvals(A- B*Kinf))
# Check if all eigenvalues are less than 1
if all(x -> x < 1.0, eig)
    println("The system is stable.")
else
    println("The system is unstable.")
end
println("eigenvalues= ",eig)

##########


In [ ]:
Random.seed!(1) 

x0= 1*X̄ # initialize the simulation with X̄

#### SIMULATION SETUP ####
# create "trajectory" of hover for the length of the simulation
X_ref= [X̄ for i= 1:N_sim]
U_ref= [Ū for i= 1:N_sim]

du_min = u_min .- Ū
du_max = u_max .- Ū

# simulation states
X_sim = [zeros(nx) for i = 1:N_sim]
X_sim[1] = 1* x0 # + randn(n_states) * 0.001

#Utarget= -B\((A- I(nx)) *X̄ + D*wind_velocity)

# # ##########################
ΔX_ref = [(X_ref[i] - X̄) for i = 1:length(X_ref)]
ΔU_ref = [(U_ref[i] - Ū) for i = 1:length(U_ref)]
# pad the end with end conditions
ΔX_ref = [ΔX_ref...,[ΔX_ref[N] for i = 1:N]...]  # Xref for MPC, padded with xgoals
ΔU_ref = [ΔU_ref..., [ΔU_ref[N-1] for i=1:N-1]...] # Uref for MPC, padded with end controls


# simulation states
X_sim = [zeros(nx) for i = 1:N_sim]
X_sim[1] = 1*x0 # + randn(n_states) * 0.001
ΔX_sim = [zeros(nx) for i = 1:N_sim]
ΔX_sim[1] = x0 - X̄ 

# simulation controls (from MPC)
U_sim = [zeros(nu) for i = 1:N_sim-1]
ΔU_sim = [zeros(nu) for i = 1:N_sim-1]

# estimated states (from Kalman)
x̂ = [zeros(nx) for i = 1:N_sim]
x̂[1] = X_sim[1]
Δx̂ = [zeros(nx) for i = 1:N_sim]
Δx̂[1] = x0- X̄

dxg= 1* ΔX_ref[N]


##############################
estimated_wind= [zeros(3) for i = 1:N_sim]
estimated_wind[1]= [0.0;0.0;0.0] #wind_velocity

#### u target stuff
#Utarget= -B\((A- I(nx)) *X̄ + wind_dynamics*wind_velocity)

wind_velocity_sim= [zeros(3) for i= 1:N_sim]

# simulate 
@showprogress "simulating" for i = 1:N_sim-1     
    #update wind
    if i>=100
        wind_velocity_sim[i] = [0.2; 0.0; 0.0]
        wind_velocity_sim[i] = wind_velocity_sim[i] .+ 0.01*randn(3)#.*wind_stdev
    end

    # given a window of N_mpc timesteps, get current reference trajectory
    ΔX_ref_horizon = ΔX_ref[i: (i+ N_mpc-1)] 

    # ΔU_ref_horizon= ΔU_ref[i: (i+ N_mpc-1)]
    # implement live Utarget updating
    Utarget= -B\((A - I(nx)) * X̄ + wind_dynamics * estimated_wind[i])
    ΔU_ref_horizon= [Utarget for i= 1:N_mpc]

    # call convex mpc controller with state estimate 
    ΔU_sim[i] = convex_mpc(A, B, wind_dynamics, ΔX_ref_horizon, ΔU_ref_horizon, Δx̂[i], dxg, du_min, du_max, N_mpc, Q, R, Qf, estimated_wind[i]) #+ randn(n_inputs) * 0.01
    U_sim[i] = ΔU_sim[i] + Ū

    ### i think if we want to simulate with the nonlinear dynamcis, we don't need the Deltas... i think we need that if we sim with the linearized dynamcis
    X_sim[i+1] = rk4(model, hexrotor_dynamics, X_sim[i], U_sim[i], wind_velocity_sim[i], dt) + wind_dynamics * wind_velocity_sim[i] #+ randn(n_states)*0.001
    ΔX_sim[i+1]= X_sim[i+1] - X̄
 
    C= I(12)
    v_k_state= 0.01 * randn(12)
    measurement= C * ΔX_sim[i+1] + v_k_state # measurement is taken from the actual simulated dynamics 

    Δx̂[i+1], estimated_wind[i+1], Σ[i+1] = kalman_filter(Δx̂[i], measurement, estimated_wind[i], ΔU_sim[i], Σ[i], A, B, wind_dynamics, W, V) 
    x̂[i+1]= Δx̂[i+1] + X̄
end



In [ ]:
# -------------plotting/animation---------------------------
Xm = mat_from_vec(X_sim)
Um = mat_from_vec(U_sim)
x̂m= mat_from_vec(x̂)
#Σm= mat_from_vec(Σ)

plot1= plot(t_vec_sim,Xm[1:3,:]',title = "Position",
xlabel = "Time (s)", ylabel = "Position (m)",
label = ["x" "y" "z"])

plot!(plot1,t_vec_sim, x̂m[1:3,:]', label=["Estimated x" "Estimated y" "Estimated z"], linestyle=:dash, legend=:right)
display(plot1)

    
plot2= plot(t_vec_sim,Xm[7:9,:]', title = "Attitude",
xlabel = "Time (s)", ylabel = "Attitude (rad)", ylimits=(-0.5,0.5),
label = ["Roll" "Pitch" "Yaw"])

plot!(plot2,t_vec_sim, x̂m[7:9,:]',label=["Estimated roll" "Estimated pitch" "Estimated yaw"], linestyle=:dash)
display(plot2)

plot3= plot(t_vec_sim,Xm[4:6,:]', title = "Velocity",
xlabel = "Time (s)", ylabel = "Velocity (m/s)",
label = ["x" "y" "z"])

plot!(plot3,t_vec_sim, x̂m[4:6,:]',label=["Estimated Vx" "Estimated Vy" "Estimated Vz"], linestyle=:dash)
display(plot3)

plot4= plot(t_vec_sim,Xm[10:12,:]', title = "Angular Velocity",
xlabel = "Time (s)", ylabel = "Angular Velocity (rad/s)",
label = ["ωθ" "ωϕ" "ωψ"], legend=:right)

plot!(plot4,t_vec_sim, x̂m[10:12,:]',label=["Estimated ωθ" "Estimated ωϕ" "Estimated ωψ"], linestyle=:dash)
display(plot4)


display(plot(t_vec_sim[1:end-1],Um',title = "Controls, u",
        xlabel = "Time (s)", ylabel= "Rotor Speed (rad/s)", label= ["Rotor 1" "Rotor 2" "Rotor 3" "Rotor 4" "Rotor 5" "Rotor 6"]))  


# mesh cat display
display(animate_hexrotor(X_sim, dt))

In [ ]:
#### error between target and simulation
error_gt_sim= [zeros(nx) for i= 1:N_sim]
state_gt= X̄
for i = 1:N_sim
    error_gt_sim[i]= abs.(state_gt.- X_sim[i])
end

# plotting
error_m= mat_from_vec(error_gt_sim)

plot5= plot(t_vec_sim, error_m[1:3,:]',
xlabel = "Time (s)", ylabel = "Error (m)",
label = ["x" "y" "z"])
display(plot5)
#title = "Position Absolute Error",

plot6= plot(t_vec_sim, error_m[7:9,:]',
xlabel = "Time (s)", ylabel = "Error (rad)",
label = ["θ" "ϕ" "ψ"])
display(plot6)
#title = "Attitude Absolute Error",

plot7= plot(t_vec_sim, error_m[4:6,:]',
xlabel = "Time (s)", ylabel = "Error (m/s)",
label = ["Vx" "Vy" "Vz"])
display(plot7)
#title = "Linear Velocity Absolute Error",
    
plot8= plot(t_vec_sim, error_m[10:12,:]',
xlabel = "Time (s)", ylabel = "Error (rad/s)",
label = ["ωθ" "ωy" "ωz"])
display(plot8)
#title = "Angular Velocity Absolute Error",

In [ ]:
#### looking at the wind state
wind_m= mat_from_vec(wind_velocity_sim)
est_wind_m= mat_from_vec(estimated_wind)

plot9= plot(t_vec_sim[1:end-1], wind_m[:,1:end-1]',
xlabel = "Time (s)", ylabel = "Wind Velocity (m/s)",
label = ["x" "y" "z"], legend=:right)
plot!(plot9,t_vec_sim[1:end-1], est_wind_m[:,1:end-1]',label=["Estimated x" "Estimated y" "Estimated z"], linestyle=:dash)
display(plot9)
#title = "Wind Disturbance Model",
